<a href="https://colab.research.google.com/github/ilyas-r27/Graph_Theory-Group_2/blob/main/Fuery-Algorithm-Undirected.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from collections import deque
from typing import List, Tuple, Dict, Any, Optional

In [ ]:
Node = Any
Edge = Tuple[Node, Node]
Adj  = Dict[Node, List[Node]]

In [ ]:
# ---------- helpers for undirected graphs ----------

def build_adj(nodes: List[Node], edges: List[Edge]) -> Adj:
    """Undirected multigraph adjacency list."""
    adj: Adj = {v: [] for v in nodes}
    for u, v in edges:
        if u not in adj: adj[u] = []
        if v not in adj: adj[v] = []
        adj[u].append(v)
        adj[v].append(u)
    return adj

def erase_edge(adj: Adj, u: Node, v: Node) -> None:
    adj[u].remove(v)
    adj[v].remove(u)

def add_edge(adj: Adj, u: Node, v: Node) -> None:
    adj[u].append(v)
    adj[v].append(u)

def reachable_count(start: Node, adj: Adj) -> int:
    """# of vertices reachable from 'start' with current edges."""
    seen = {start}
    q = deque([start])
    while q:
        x = q.popleft()
        for y in adj[x]:
            if y not in seen:
                seen.add(y)
                q.append(y)
    return len(seen)

def has_isolated(adj: Adj) -> bool:
    return any(len(nbrs) == 0 for nbrs in adj.values())

def connected_over_all_nodes(adj: Adj) -> bool:
    """Single connected component over listed nodes & at least one edge."""
    m = sum(len(n) for n in adj.values()) // 2
    if m == 0:  # no edges → not Eulerian for this assignment
        return False
    if has_isolated(adj):  # disallow isolated vertices
        return False
    start = sorted(adj.keys())[0]
    return reachable_count(start, adj) == len(adj)

def odd_vertices(adj: Adj) -> List[Node]:
    return sorted([v for v, nbrs in adj.items() if len(nbrs) % 2 == 1])

def is_bridge(adj: Adj, u: Node, v: Node) -> bool:
    """(u, v) is a bridge if removing it reduces reachability from u."""
    before = reachable_count(u, adj)
    erase_edge(adj, u, v)
    after = reachable_count(u, adj) if adj[u] else 0
    add_edge(adj, u, v)
    return after < before

In [ ]:
# ---------- Fleury (undirected) ----------

def fleury_euler_path(nodes: List[Node], edges: List[Edge]) -> Optional[List[Node]]:
    """Return Eulerian path as list of nodes; None if it doesn't exist."""
    adj = build_adj(nodes, edges)

    # Preconditions: connectivity + parity
    if not connected_over_all_nodes(adj):
        return None
    odd = odd_vertices(adj)
    if len(odd) not in (0, 2):
        return None

    # Starting point
    cur = min(odd) if len(odd) == 2 else min([v for v in adj if len(adj[v]) > 0])
    path = [cur]
    edges_left = sum(len(n) for n in adj.values()) // 2

    while edges_left > 0:
        nbrs = sorted(adj[cur])
        if not nbrs:
            return None  # stuck early
        # Prefer a non-bridge when there's a choice
        nxt = None
        if len(nbrs) == 1:
            nxt = nbrs[0]
        else:
            for w in nbrs:
                if not is_bridge(adj, cur, w):
                    nxt = w
                    break
            if nxt is None:  # all options are bridges
                nxt = nbrs[0]
        erase_edge(adj, cur, nxt)
        edges_left -= 1
        cur = nxt
        path.append(cur)

    return path

def print_euler_result(nodes: List[Node], edges: List[Edge]) -> None:
    ans = fleury_euler_path(nodes, edges)
    if ans is None:
        print("euler path not found")
    else:
        print("Eulerian path:", "-".join(map(str, ans)))

In [ ]:
print("Example 1")
nodes1 = [0, 1, 2, 3]
edges1 = [(0,1), (0,2), (1,2), (2,3)]
print_euler_result(nodes1, edges1)  # expected: Eulerian path: 2-0-1-2-3

In [ ]:
print("\nExample 2")
nodes2 = [0, 1, 2, 3]
edges2 = [(0,1), (1,2)]
print_euler_result(nodes2, edges2)  # expected: euler path not found